In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error


train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')

In [ ]:
# Get unique values from the 'family' column
unique_families = train['family'].unique()


print("Unique values in the 'family' column:")
print(unique_families)


num_unique_families = len(unique_families)
print(f"\nNumber of unique families: {num_unique_families}")

Unique values in the 'family' column:
['AUTOMOTIVE' 'BABY CARE' 'BEAUTY' 'BEVERAGES' 'BOOKS' 'BREAD/BAKERY'
 'CELEBRATION' 'CLEANING' 'DAIRY' 'DELI' 'EGGS' 'FROZEN FOODS' 'GROCERY I'
 'GROCERY II' 'HARDWARE' 'HOME AND KITCHEN I' 'HOME AND KITCHEN II'
 'HOME APPLIANCES' 'HOME CARE' 'LADIESWEAR' 'LAWN AND GARDEN' 'LINGERIE'
 'LIQUOR,WINE,BEER' 'MAGAZINES' 'MEATS' 'PERSONAL CARE' 'PET SUPPLIES'
 'PLAYERS AND ELECTRONICS' 'POULTRY' 'PREPARED FOODS' 'PRODUCE'
 'SCHOOL AND OFFICE SUPPLIES' 'SEAFOOD']

Number of unique families: 33


In [ ]:
# List of unique families
unique_families = [
    'AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS', 'BREAD/BAKERY',
    'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 'GROCERY I',
    'GROCERY II', 'HARDWARE', 'HOME AND KITCHEN I', 'HOME AND KITCHEN II',
    'HOME APPLIANCES', 'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
    'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE', 'PET SUPPLIES',
    'PLAYERS AND ELECTRONICS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE',
    'SCHOOL AND OFFICE SUPPLIES', 'SEAFOOD'
]

# Dictionary to store average sales for each family
average_sales = {}


for family in unique_families:
    avg_sales = train.loc[train['family'] == family, 'sales'].mean()
    average_sales[family] = avg_sales

# Convert the dictionary to a DataFrame 
average_sales_df = pd.DataFrame(list(average_sales.items()), columns=['Family', 'Average Sales'])


print(average_sales_df)


                        Family  Average Sales
0                   AUTOMOTIVE       6.101236
1                    BABY CARE       0.110528
2                       BEAUTY       3.715723
3                    BEVERAGES    2385.793151
4                        BOOKS       0.070797
5                 BREAD/BAKERY     463.336254
6                  CELEBRATION       8.370469
7                     CLEANING    1072.416744
8                        DAIRY     709.154889
9                         DELI     265.135067
10                        EGGS     171.420516
11                FROZEN FOODS     154.766954
12                   GROCERY I    3776.972100
13                  GROCERY II      21.584048
14                    HARDWARE       1.137833
15          HOME AND KITCHEN I      20.470342
16         HOME AND KITCHEN II      16.722420
17             HOME APPLIANCES       0.457476
18                   HOME CARE     176.198029
19                  LADIESWEAR       7.160629
20             LAWN AND GARDEN    

# Linear Regression with feature engineer

In [36]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
train['date'] = pd.to_datetime(train['date'])
train['is_1st_Jan'] = ((train['date'].dt.month == 1) & (train['date'].dt.day == 1)).astype(int)

is_closed = {'store_nbr': [], 'closed_1st_Jan': [], 'open_date': []}

for store in train['store_nbr'].unique():
    closed_on_jan1 = (train['is_1st_Jan'] > 0) & (train['store_nbr'] == store)
    is_closed['store_nbr'].append(store)
    is_closed['closed_1st_Jan'].append(np.sum(train['sales'][closed_on_jan1]) < 0.1)
    is_closed['open_date'].append(np.min(train['date'][closed_on_jan1]))

is_closed_df = pd.DataFrame(is_closed)

train = pd.merge(train, is_closed_df, on='store_nbr', how='left')
train['is_closed'] = (train['date'] < train['open_date']) | (train['is_1st_Jan'] & train['closed_1st_Jan'])

if 'is_sales' in train:
    train.drop(['is_sales'], axis=1, inplace=True)
    
df_grouped = (train.groupby(['store_nbr', 'family'])
              .rolling(window=30, on='date')['sales']
              .sum()
              .bfill()
              .reset_index())

df_grouped['is_sales'] = df_grouped['sales'] > 0
df_grouped.drop(['sales'], axis=1, inplace=True)

train = pd.merge(train, df_grouped, how='left')
train['is_sales'] = train['is_sales'] & (~train['is_closed'])

books_sales_index = (train['family'] == 'BOOKS') & (train['sales'] > 0)
date_books_start = np.min(train['date'][books_sales_index])
average_books_sales = np.mean(train['sales'][books_sales_index])

train['is_sales'] = np.where(train['family'] == 'BOOKS', 0, train['is_sales'])

lr = LinearRegression()

def get_category_inflation(df_grouped, train_ind):
    """
    Calculates category-specific inflation trends based on a linear regression model 
    applied to each unique 'family' category in the dataset.

    Parameters:
    df_grouped : pd.DataFrame
        DataFrame containing sales data. It must include the columns 'day_n', 
        'family', and 'sales'.
    train_ind : np.ndarray or pd.Series
        Boolean array or Series indicating the training data rows.

    Returns:
    pd.DataFrame
        A DataFrame containing columns 'day_n', 'family', and the calculated
        'cat_inflation'.
    """
    inflation_values = np.full(df_grouped.shape[0], np.mean(df_grouped['sales']))
    
    for family in df_grouped['family'].unique():
        family_mask = (df_grouped['family'] == family)
        train_mask = family_mask & train_ind
        
        if not np.any(train_mask):
            continue
        
        lr.fit(df_grouped[['day_n']][train_mask], df_grouped['sales'][train_mask])
        inflation_values[family_mask] = lr.predict(df_grouped[['day_n']][family_mask])

    df_grouped['cat_inflation'] = inflation_values
    return df_grouped[['day_n', 'family', 'cat_inflation']]

train['day_n'] = (train['date'] - train['date'].min()).dt.days
df_grouped = train[train['is_sales'] == 1].groupby(['day_n', 'family'])[['sales']].mean().reset_index()

train_ind = df_grouped['day_n'] < 1350
df_grouped = get_category_inflation(df_grouped, train_ind)

train = pd.merge(train, df_grouped, on=['day_n', 'family'], how='left')
train['pred'] = train['cat_inflation'].fillna(0)
train['pred'] = np.where(train['is_sales'] == 0, 0, train['pred'])
train['pred'] = np.maximum(train['pred'], 0)

train_ind = train['day_n'] < 1350
test_ind = ~train_ind
rmsle = np.sqrt(mean_squared_log_error(train['sales'][test_ind], train['pred'][test_ind]))
print(f"Root Mean Squared Log Error: {rmsle:.4f}")

is_sales_test = train[['store_nbr', 'family', 'is_sales']][train['date'] == '2017-08-15']

test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
test['date'] = pd.to_datetime(test['date'])
test['day_n'] = (test['date'] - train['date'].min()).dt.days
test = pd.merge(test, is_sales_test, how='left', on=['store_nbr', 'family'])

df_avg_sales = train[train['is_sales'] == 1].groupby(['day_n', 'family'])[['sales']].mean().reset_index()
df_test_sales = test[test['is_sales'] == 1][['day_n', 'family']].drop_duplicates()
df_combined = pd.concat([df_avg_sales, df_test_sales])

train_ind = df_combined['day_n'] < df_test_sales['day_n'].min()

df_combined = get_category_inflation(df_combined, train_ind)

test = pd.merge(test, df_combined, how='left')
test['sales'] = np.where(test['is_sales'] == 0, 0, test['cat_inflation'])
test['sales'] = test['sales'].fillna(0)

submission = test[['id', 'sales']].drop_duplicates()
submission.to_csv('submission_linear.csv', index=False)

Root Mean Squared Log Error: 0.9516


# Linear Regression without feature engineer

In [37]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
train['date'] = pd.to_datetime(train['date'])

df_grouped = (train.groupby(['store_nbr', 'family'])
              .rolling(window=30, on='date')['sales']
              .sum()
              .bfill()
              .reset_index())

df_grouped['is_sales'] = df_grouped['sales'] > 0
df_grouped.drop(['sales'], axis=1, inplace=True)

train = pd.merge(train, df_grouped, how='left')

books_sales_index = (train['family'] == 'BOOKS') & (train['sales'] > 0)
date_books_start = np.min(train['date'][books_sales_index])
average_books_sales = np.mean(train['sales'][books_sales_index])

train['is_sales'] = np.where(train['family'] == 'BOOKS', 0, train['is_sales'])

lr = LinearRegression()

def get_category_inflation(df_grouped, train_ind):
    """
    Calculates category-specific inflation trends based on a linear regression model 
    applied to each unique 'family' category in the dataset.

    Parameters:
    df_grouped : pd.DataFrame
        DataFrame containing sales data. It must include the columns 'day_n', 
        'family', and 'sales'.
    train_ind : np.ndarray or pd.Series
        Boolean array or Series indicating the training data rows.

    Returns:
    pd.DataFrame
        A DataFrame containing columns 'day_n', 'family', and the calculated
        'cat_inflation'.
    """
    inflation_values = np.full(df_grouped.shape[0], np.mean(df_grouped['sales']))

    for family in df_grouped['family'].unique():
        family_mask = (df_grouped['family'] == family)
        train_mask = family_mask & train_ind
        
        if not np.any(train_mask):
            continue
        
        lr.fit(df_grouped[['day_n']][train_mask], df_grouped['sales'][train_mask])
        inflation_values[family_mask] = lr.predict(df_grouped[['day_n']][family_mask])

    df_grouped['cat_inflation'] = inflation_values
    return df_grouped[['day_n', 'family', 'cat_inflation']]

train['day_n'] = (train['date'] - train['date'].min()).dt.days
df_grouped = train[train['is_sales'] == 1].groupby(['day_n', 'family'])[['sales']].mean().reset_index()

train_ind = df_grouped['day_n'] < 1350
df_grouped = get_category_inflation(df_grouped, train_ind)

train = pd.merge(train, df_grouped, on=['day_n', 'family'], how='left')
train['pred'] = train['cat_inflation'].fillna(0)
train['pred'] = np.where(train['is_sales'] == 0, 0, train['pred'])
train['pred'] = np.maximum(train['pred'], 0)

train_ind = train['day_n'] < 1350
test_ind = ~train_ind
rmsle = np.sqrt(mean_squared_log_error(train['sales'][test_ind], train['pred'][test_ind]))
print(f"Root Mean Squared Log Error: {rmsle:.4f}")

is_sales_test = train[['store_nbr', 'family', 'is_sales']][train['date'] == '2017-08-15']

test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
test['date'] = pd.to_datetime(test['date'])
test['day_n'] = (test['date'] - train['date'].min()).dt.days
test = pd.merge(test, is_sales_test, how='left', on=['store_nbr', 'family'])

df_avg_sales = train[train['is_sales'] == 1].groupby(['day_n', 'family'])[['sales']].mean().reset_index()
df_test_sales = test[test['is_sales'] == 1][['day_n', 'family']].drop_duplicates()
df_combined = pd.concat([df_avg_sales, df_test_sales])
train_ind = df_combined['day_n'] < df_test_sales['day_n'].min()

df_combined = get_category_inflation(df_combined, train_ind)

test = pd.merge(test, df_combined, how='left')
test['sales'] = np.where(test['is_sales'] == 0, 0, test['cat_inflation'])
test['sales'] = test['sales'].fillna(0)

submission = test[['id', 'sales']].drop_duplicates()
submission.to_csv('submission_without.csv', index=False)


Root Mean Squared Log Error: 0.9823


#  Random Forest Regressor 

In [ ]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

# Step 2: Load Data
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')

# Step 3: Merge Datasets
def merge_data(df):
    df = df.merge(stores, on='store_nbr', how='left')
    df = df.merge(oil, on='date', how='left')
    df = df.merge(holidays, on='date', how='left')
    return df

train = merge_data(train)
test = merge_data(test)

# Step 4: Handle Missing Values
def fill_missing_values(df):
    df = df.copy()  # Avoid chained assignment issues
    df['onpromotion'] = df['onpromotion'].fillna(0)
    df['dcoilwtico'] = df['dcoilwtico'].ffill()
    return df

train = fill_missing_values(train)
test = fill_missing_values(test)

# Step 5: Feature Engineering
def create_features(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek
    return df

train = create_features(train)
test = create_features(test)

# Step 6: Aggregate Sales Data
train_grouped = train.groupby(
    ['store_nbr', 'family', 'year', 'month', 'day', 'day_of_week', 'onpromotion', 'dcoilwtico'], 
    as_index=False
)['sales'].sum()

# Step 7: Encoding Categorical Features
train_encoded = pd.get_dummies(train_grouped, columns=['family'], drop_first=True)

X = train_encoded.drop('sales', axis=1)
y = train_encoded['sales']

# Step 8: K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmsle_scores = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Step 9: Train the Model
    model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)

    # Step 10: Evaluate Model Performance
    y_pred = model.predict(X_val)
    rmsle = np.sqrt(mean_squared_log_error(y_val, np.maximum(0, y_pred)))  # Avoid negative predictions
    rmsle_scores.append(rmsle)

print(f'Average RMSLE (K-Fold CV): {np.mean(rmsle_scores):.4f}')

# Step 11: Prepare Test Data for Prediction
test_encoded = pd.get_dummies(test, columns=['family'], drop_first=True)

X_test = test_encoded.reindex(columns=X.columns, fill_value=0)

if 'id' in X_test.columns:
    X_test = X_test.drop('id', axis=1)

# Step 12: Train the Model on the Entire Training Set for Final Predictions
model.fit(X, y)  # Fit model on the entire training data

# Step 13: Predict on Test Set
test_preds = model.predict(X_test)

# Step 14: Create Submission File
submission = pd.DataFrame({
    'id': test['id'],
    'sales': np.maximum(0, test_preds)  # Ensure non-negative predictions
})
submission.to_csv('submission_3.csv', index=False)


Data Exploration and Preparation in Random Forest

In [16]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

# Step 2: Load Data
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')

In [17]:
def merge_data(df):
    df = df.merge(stores, on='store_nbr', how='left')
    df = df.merge(oil, on='date', how='left')
    df = df.merge(holidays, on='date', how='left')
    return df

train = merge_data(train)
test = merge_data(test)

print("Merged Train Dataset Structure:")
print(train.head(), "\n")

Merged Train Dataset Structure:
   id        date  store_nbr      family  sales  onpromotion   city  \
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0  Quito   
1   1  2013-01-01          1   BABY CARE    0.0            0  Quito   
2   2  2013-01-01          1      BEAUTY    0.0            0  Quito   
3   3  2013-01-01          1   BEVERAGES    0.0            0  Quito   
4   4  2013-01-01          1       BOOKS    0.0            0  Quito   

       state type_x  cluster  dcoilwtico   type_y    locale locale_name  \
0  Pichincha      D       13         NaN  Holiday  National     Ecuador   
1  Pichincha      D       13         NaN  Holiday  National     Ecuador   
2  Pichincha      D       13         NaN  Holiday  National     Ecuador   
3  Pichincha      D       13         NaN  Holiday  National     Ecuador   
4  Pichincha      D       13         NaN  Holiday  National     Ecuador   

          description transferred  
0  Primer dia del ano       False  
1  Primer dia del 

In [33]:
def fill_missing_values(df):
    df = df.copy()
    df['onpromotion'] = df['onpromotion'].fillna(0)
    df['dcoilwtico'] = df['dcoilwtico'].ffill()
    return df

train = fill_missing_values(train)
test = fill_missing_values(test)


In [35]:
def create_features(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek
    return df

train = create_features(train)
test = create_features(test)

In [30]:
train_grouped = train.groupby(
    ['store_nbr', 'family', 'year', 'month', 'day', 'day_of_week', 'onpromotion', 'dcoilwtico'], 
    as_index=False
)['sales'].sum()


In [21]:
train_encoded = pd.get_dummies(train_grouped, columns=['family'], drop_first=True)

# Print encoded train structure
print("Encoded Train Dataset Structure:")
print(train_encoded.head(), "\n")

Encoded Train Dataset Structure:
   store_nbr  year  month  day  day_of_week  onpromotion  dcoilwtico  sales  \
0          1  2013      1    2            2            0       93.14    2.0   
1          1  2013      1    3            3            0       92.97    3.0   
2          1  2013      1    4            4            0       93.12    3.0   
3          1  2013      1    5            5            0       93.12    5.0   
4          1  2013      1    6            6            0       93.12    2.0   

   family_BABY CARE  family_BEAUTY  ...  family_MAGAZINES  family_MEATS  \
0             False          False  ...             False         False   
1             False          False  ...             False         False   
2             False          False  ...             False         False   
3             False          False  ...             False         False   
4             False          False  ...             False         False   

   family_PERSONAL CARE  family_PET SUPPL